<a href="https://colab.research.google.com/github/hyamynl619/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_and_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import seaborn as sns
from matplotlib import style



In [0]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

--2019-12-10 08:13:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.2’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2019-12-10 08:13:23 (137 KB/s) - ‘house-votes-84.data.2’ saved [18171/18171]



In [0]:
df = pd.read_csv('house-votes-84.data', 
                 header=None,
                  names=['Class Name', 'Handicapped-Infants', 'Water-Project', 'Budget', 'Physician-Fee-Freeze',
                 'El-Salvador-Aid','Religious-Groups', 'Anti-Satellite-Ban','Aid-To-Contras','Mx-Missle',
                 'Immigration','Synfuels','Education','Right-to-Sue','Crime','Duty-Free','South-Africa'])

print(df.shape)
df.head()

(435, 17)


,Class Name,Handicapped-Infants,Water-Project,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious-Groups,Anti-Satellite-Ban,Aid-To-Contras,Mx-Missle,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df.isnull().sum()

Class Name              0
Handicapped-Infants     0
Water-Project           0
Budget                  0
Physician-Fee-Freeze    0
El-Salvador-Aid         0
Religious-Groups        0
Anti-Satellite-Ban      0
Aid-To-Contras          0
Mx-Missle               0
Immigration             0
Synfuels                0
Education               0
Right-to-Sue            0
Crime                   0
Duty-Free               0
South-Africa            0
dtype: int64

In [0]:
df = df.replace({'?':np.NaN, 'n':0, 'y':1})
df.head()

,Class Name,Handicapped-Infants,Water-Project,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious-Groups,Anti-Satellite-Ban,Aid-To-Contras,Mx-Missle,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
df.isnull().sum()

Class Name                0
Handicapped-Infants      12
Water-Project            48
Budget                   11
Physician-Fee-Freeze     11
El-Salvador-Aid          15
Religious-Groups         11
Anti-Satellite-Ban       14
Aid-To-Contras           15
Mx-Missle                22
Immigration               7
Synfuels                 21
Education                31
Right-to-Sue             25
Crime                    17
Duty-Free                28
South-Africa            104
dtype: int64

In [0]:
dem = df[df['Class Name'] == 'democrat']
rep = df[df['Class Name'] == 'republican']

dem.describe()

,Handicapped-Infants,Water-Project,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious-Groups,Anti-Satellite-Ban,Aid-To-Contras,Mx-Missle,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
rep.describe()

,Handicapped-Infants,Water-Project,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious-Groups,Anti-Satellite-Ban,Aid-To-Contras,Mx-Missle,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
col = rep['Immigration']
rep_immi = col[-np.isnan(col)]

rep_immi

0      1.0
1      0.0
7      0.0
8      0.0
10     0.0
      ... 
420    1.0
427    1.0
430    1.0
432    0.0
434    1.0
Name: Immigration, Length: 165, dtype: float64

In [0]:
rep_immi.sum()/len(rep_immi)

0.5575757575757576

In [0]:
col2 = dem['Immigration']
dem_immi = col2[-np.isnan(col2)]

dem_immi

2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
      ... 
425    1.0
426    1.0
428    1.0
429    1.0
431    1.0
Name: Immigration, Length: 263, dtype: float64

In [0]:
rep['Immigration'].mean()

0.5575757575757576

In [0]:
dem['Immigration'].mean()

0.4714828897338403

In [0]:
dem_immi.sum()/len(dem_immi)

0.4714828897338403

1) Null Hypothesis: Democrat votes will be higher than republican votes for the 'Immigration' bill. 

2) Alternative Hypothesis: 
The votes will be equal to each other. 

Levels of support between the two parties will not differ.

3)95% Confidence level



In [0]:
ttest_ind(rep['Immigration'], dem['Immigration'], nan_policy='omit')

Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

In [0]:
print(len(rep_immi))
print(len(dem_immi))

165
263


4) t-statistics = 1.735
5) p-value = 0.083
 *more dem votes than rep p-value <0.1

Conclusion: due to the p-value of 0.083 reject my Null Hypothesis.

1) Null Hypothesis: The Republican votes will be equal to the democratic votes on the 'Crime' bill.

2)Alternative Hypothesis: Democratic votes will be higher than the Republican votes.

3.Confidence Level: 95% 

In [0]:
col3 = dem['Crime']
dem_crime = col3[-np.isnan(col3)]

dem_crime

2      1.0
3      0.0
4      1.0
5      1.0
6      1.0
      ... 
424    1.0
425    0.0
426    0.0
428    0.0
431    0.0
Name: Crime, Length: 257, dtype: float64

In [0]:
col4 = rep['Crime']
rep_crime = col4[-np.isnan(col4)]

rep_crime

0      1.0
1      1.0
7      1.0
8      1.0
10     1.0
      ... 
427    1.0
430    1.0
432    1.0
433    1.0
434    1.0
Name: Crime, Length: 161, dtype: float64

In [0]:
rep_crime.mean()

0.9813664596273292

In [0]:
dem_crime.mean()

0.35019455252918286

In [0]:
rep_crime.sum()/len(rep_crime)

0.9813664596273292

In [0]:
dem_crime.sum()/len(dem_crime)

0.35019455252918286

In [0]:
ttest_ind(dem['Crime'], rep['Crime'], nan_policy='omit')

Ttest_indResult(statistic=-16.342085656197696, pvalue=9.952342705606092e-47)

In [0]:
print(len(rep_crime))
print(len(dem_crime))

161
257


4) t-statistics= -16.34
5) p-value= 9.95

6) Conclusion: Due to the p-value being 9.95 I reject the Null Hypothesis of the vote being equal between the parties. 

In [0]:
rep['Water-Project'].mean()

0.5067567567567568

In [0]:
dem['Water-Project'].mean()

0.502092050209205

1) Null Hypothesis: The votes between the two parties will be equal for the 'Water-Project' bill.

2) Alternative Hypothesis: The votes are not equal to each other.

3) Confidence Level: 95%

In [0]:
col5 = rep['Water-Project']

rep_water= col5[-np.isnan(col5)]
rep_water

0      1.0
1      1.0
7      1.0
8      1.0
10     1.0
      ... 
420    1.0
427    0.0
430    0.0
433    0.0
434    1.0
Name: Water-Project, Length: 148, dtype: float64

In [0]:
col6 = dem['Water-Project']

dem_water= col6[-np.isnan(col6)]
dem_water

2      1.0
3      1.0
4      1.0
5      1.0
6      1.0
      ... 
424    1.0
425    0.0
426    0.0
429    0.0
431    0.0
Name: Water-Project, Length: 239, dtype: float64

In [0]:
print(dem_water.mean())
rep_water.mean()

0.502092050209205


0.5067567567567568

In [0]:
print(len(rep_water))
print(len(dem_water))

148
239


In [0]:
ttest_ind(dem['Water-Project'], rep['Water-Project'], nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

4)t-statistics= -0.088
5)p-value= 0.929

Conclusion: Due to the p-value being 0.929 I failed to reject my Null Hypothesis. The votes are similar to eachother between both parties. 